# Goldsmith

 - Loads the Goldsmith file (MSS12) and exports peaksets

In [1]:
import sys,glob
sys.path.append('/home/simon/git/burns/code/')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/'
blank_mz_tol = 5
mz_tol = 5

In [4]:
group1 = ['MSS12']
files1 = []
for g in group1:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files1.append((match,data_dir+bfilename))
group1,paper1 = zip(*files1)
group1 = paper1

blank_patterns = 'Blank_*'
blank_files = glob.glob(data_dir + blank_patterns)

In [5]:
from load_di import load_di

In [6]:
data = {}
for f in files1:
    out_file = '../csv_files/' + f[0].split('/')[-1].split('.')[0]+'.csv'
    data[f[0]] = load_di(f[0],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)
    data[f[1]] = load_di(f[1],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)
    
for b in blank_files:
    out_file = '../csv_files/' + b.split('/')[-1].split('.')[0]+'.csv'
    data[b] = load_di(b,out_file = out_file,normalise = None,mz_tol = blank_mz_tol)


In [7]:
from matching import Greedy
import csv
import numpy as np
options = {'log':'log','normalise':'tus','missing':1e-5}
if options['normalise'] == 'tus':
    normalised_data = {}
    for f,spec in data.items():
        mz_list,i_list = zip(*spec)
        tot_intensity = sum(i_list)
        new_intensity = [1000.0*i/tot_intensity for i in i_list]
        new_spec = zip(mz_list,new_intensity)
        normalised_data[f] = new_spec
elif options['normalise'] == 'none':
    normalised_data = data
    
g = Greedy()
mp = g.process(normalised_data,mz_tol)

display_names = {}
for g in group1:
    display_names[g] = g.split('/')[-1]

all_names = list(group1)
res = []


heads = ["mz"] + [display_names[a] for a in all_names] + ["n_blank"]
for ps in mp:
    row = [ps.mean_mz]
    g1 = []
    for n in all_names:
        peak = filter(lambda x: x[2] == n,ps.peaks)
        if len(peak) == 0:
            row.append("nan")
            # take max intensity
            peak = sorted(peak,key = lambda x: x[1], reverse = True)
        else:
            row.append(peak[0][1])
        if n in group1:
            g1.append(row[-1])

    ng1 = sum([1 for a in g1 if not a == "nan"])
    if options['missing'] == 'nan':
        g1 = [a for a in g1 if not a == "nan"]
    else:
        g1 = [a if not a == "nan" else options["missing"] for a in g1]
    if options['log'] == 'log':
        g1 = [np.log10(a) for a in g1]


    b_count = 0
    for n in blank_files:
        peak = filter(lambda x: x[2] == n,ps.peaks)
        if len(peak) > 0:
            b_count += 1
            
    row.append(b_count)
    res.append(row)

# Sort by number present
res = sorted(res,key = lambda x: sum([1 if l == 'nan' else 0 for l in x]))
# remove those only present in the blanks
res = filter(lambda x: sum([1 if l == 'nan' else 0 for l in x])<4,res)
    
with open("spreadsheets/goldsmith/goldsmith_{}_{}_{}.csv".format(options['log'],options['missing'],options['normalise']),'w') as f:
    writer = csv.writer(f)
    writer.writerow(heads)
    for r in res:
        writer.writerow(r)

7235 peaks
